In [1]:
import sys ,os 
import numpy as np
import matplotlib.pyplot as plt
import struct
from PIL import Image
import pandas as pd 
import math

x_train = np.load("x_train_number.npy") 
y_train = np.load("y_train_number.npy")
x_test  = np.load("x_test_number.npy")
y_test  = np.load("y_test_number.npy")
x_train = x_train.reshape(60000,1,28,28)/255.0
y_train = y_train.T
x_test = x_test.reshape(10000,1,28,28)/255.0
y_test = y_test.T

import sys ,os 
import numpy as np
import matplotlib.pyplot as plt
import struct
from PIL import Image
import pandas as pd 
import math
# 激活函数
def relu(x):
    x_1 = np.where(x < 0 ,0,x)
    return x_1

def softmax(x):
    exp_a = np.exp(x-np.max(x,axis = 1,keepdims=True)) # 行
    sum_exp_a = np.sum(exp_a,axis = 1,keepdims=True)
    ret = exp_a/sum_exp_a
    return ret

def cross_entropy(y,y_hat):
    return -np.sum(y*np.log(y_hat + np.exp(-7)))

def get_im2col_indices(x_shape, field_height, field_width, stride=1,padding=0):
  # First figure out what the size of the output should be
  N, C, H, W = x_shape
  assert (H + 2 * padding - field_height) % stride == 0
  assert (W + 2 * padding - field_height) % stride == 0
  out_height = int((H + 2 * padding - field_height) / stride + 1)
  out_width = int((W + 2 * padding - field_width) / stride + 1)

  i0 = np.repeat(np.arange(field_height), field_width)
  i0 = np.tile(i0, C)
  i1 = stride * np.repeat(np.arange(out_height), out_width)
  j0 = np.tile(np.arange(field_width), field_height * C)
  j1 = stride * np.tile(np.arange(out_width), out_height)
  i = i0.reshape(-1, 1) + i1.reshape(1, -1)
  j = j0.reshape(-1, 1) + j1.reshape(1, -1)

  k = np.repeat(np.arange(C), field_height * field_width).reshape(-1, 1)

  return (k, i, j)


def im2col(x, field_height, field_width, stride=1 ,padding=0):
  """ An implementation of im2col based on some fancy indexing """
  # Zero-pad the input
  p = padding
  x_padded = np.pad(x, ((0, 0), (0, 0), (p, p), (p, p)), mode='constant')

  k, i, j = get_im2col_indices(x.shape, field_height, field_width, stride, padding)

  cols = x_padded[:, k, i, j]
  C = x.shape[1]
  cols = cols.transpose(1, 2, 0).reshape(field_height * field_width * C, -1)
  return cols


def col2im(cols, x_shape, field_height, field_width, stride=1, padding=0):
  """ An implementation of col2im based on fancy indexing and np.add.at """
  N, C, H, W = x_shape
  H_padded, W_padded = H + 2 * padding, W + 2 * padding
  x_padded = np.zeros((N, C, H_padded, W_padded), dtype=cols.dtype)
  k, i, j = get_im2col_indices(x_shape, field_height, field_width, stride, padding)
  cols_reshaped = cols.reshape(C * field_height * field_width, -1, N)
  cols_reshaped = cols_reshaped.transpose(2, 0, 1)
  np.add.at(x_padded, (slice(None), k, i, j), cols_reshaped)
  if padding == 0:
    return x_padded
  return x_padded[:, :, padding:-padding, padding:-padding]

pass


# 卷积层
class Convolution:
    def __init__(self, w, bias, stride = 1, pad = 0):
        self.x = None
        self.w = w
        self.b  = bias
        self.stride = stride
        self.pad = pad

        self.dw = None
        self.db = None

    def conv(self, x, w, bias, stride, pad):
        #获取输入x、卷积核的维度
        FN,NC,FH,FW = w.shape
        m,nc_in, h_in,w_in = x.shape
    
        #计算输出的height和width
        h_out = int((h_in+2*pad-FH)/stride + 1)
        w_out = int((w_in+2*pad-FW)/stride + 1)

        #将x按照行展开（m*h_out*w_out,FH*FW*NC)
        #此时不同的sample同一个channel的值是相邻的
        x_col = im2col(x,FH,FW,stride,pad).T
        
        #将卷积核按列展开(FH*FW*C,FN)
        w_col = w.reshape(FN,FH*FW*NC).T

        #计算卷积 (m*h_out*w_out,FN)->T->reshape
        out = (np.dot(x_col,w_col)+bias)
       
        #将batch提到最高维度（共三维）
        out_1 = np.zeros((m,h_out*w_out,FN))
        for i in range(m*h_out*w_out):
            out_1[i%m,i//m,:] = out[i,:]
        #最后调整维度
        out = out_1.transpose((0,2,1)).reshape(m,FN,h_out,w_out)
        return out,x_col
    
    def central_symmetry(self,w):
        w_r=w.transpose((2,3,0,1))
        w_r = np.flipud(np.fliplr(w_r))
        w_r=w_r.transpose((3,2,0,1))# 2,3,0,1 交换前两个，因为这时候的卷积核个数变了
        return w_r

        
    def forward (self,x):
        self.x = x # x并不固定，不在初始化的时候直接赋值
        out,x_col = Convolution.conv(self,x,self.w,self.b,self.stride,self.pad)
        self.out_shape = out.shape
        self.x_col = x_col
        return out

    
    def backward(self, dout, learning_rate = 0.01):
        # shape为：（h_out*w_out*m，nc_out）不同batch的相邻的channel相邻
        dout_col = dout.transpose(1,0,2,3).reshape(self.w.shape[0],-1).T

        #dx dx是正确的，但是dx应该在w更新之前更新
        w_r = Convolution.central_symmetry(self,self.w)
        bias_z = np.zeros((1,self.w.shape[1]))
        dx,_ = Convolution.conv(self,dout,w_r,bias_z,1,w_r.shape[3]- 1) # padding

        dw = dout.transpose(1, 2, 3, 0).reshape(self.w.shape[0], -1).dot(self.x_col).reshape(self.w.shape)
        self.w = self.w - learning_rate*dw /float(dout.shape[0])

        db = np.sum(dout_col,axis=0,keepdims=True).reshape(1,-1)
        self.b = self.b - learning_rate*db /float(dout.shape[0])

        return dx



# 池化层
class Pooling:
    def __init__(self,pool_h,pool_w,stride = 1,pad = 0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad 
        self.x = None
        self.arg_max = None
        self.x_col = None
        
    def forward ( self,x) :
        self.x = x
        N, C, H, W = x.shape
        h_out = int(1+(H-self.pool_h)/self.stride)
        w_out = int(1+(W-self.pool_w)/self.stride)

        # 展开 (N*h_out*w_out,H*W*C)
        x_col = im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)

        # 以N为最高维reshape
        x_col_r = np.zeros((N,h_out*w_out,self.pool_h*self.pool_w*C))
        for i in range(N*h_out*w_out):
            x_col_r[i%N,i//N,:]=x_col[:,i] # 用的是x_col的转置
        x_col = x_col_r.transpose(0,2,1)

        # 以channel为最高维reshape
        x_col = x_col.reshape(N,C,self.pool_w*self.pool_h,h_out*w_out).transpose(0,1,3,2)
        self.x_col = x_col 
        # 最大值
        arg_max = np.argmax(x_col,axis=3)
        self.arg_max = arg_max 
        out = np.max(x_col,axis=3)
        out =out.reshape(N,C,h_out,w_out)
        return out
    
    def backward(self ,dout):
 
        N, C, H, W = self.x.shape
        # 根据arg_max生成二维damx
        pool_size = self.pool_h*self.pool_w
        dmax = np.zeros((dout.size,pool_size)) 
        dmax[np.arange(self.arg_max.size),self.arg_max.flatten()] = dout.flatten()

        # dmax矩阵中，不同batch分开存放，为使用col2im将其变为在一起（类似im2col结果）
        dmax_1 = np.zeros((dmax.shape))
        size = dmax.shape[0] // N
        for i in range(dmax.shape[0]):
            dmax_1[(i%size) * N + i//size,:] = dmax[i,:]
        dmax = dmax_1

        # 为和im2col的情况一样，将二维d_max中按列连接的不同channel数据分开放在不同列中
        dmax_2 = np.zeros((dmax.shape[0] //C , dmax.shape[1] *C))
        for i in range(C):
            dmax_2[:,i*dmax.shape[1]:(i+1)*dmax.shape[1]] = dmax[i*dmax_2.shape[0]:(i+1)*dmax_2.shape[0],:]
        dmax = dmax_2.T # 注意要转置
        
        # 获得dx
        dx = col2im(dmax,self.x.shape,self.pool_h,self.pool_w,self.stride,self.pad)
        return dx


# relu层
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self ,x):
        self.mask = x <= 0
        out = x
        out[self.mask] = 0
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout 
        return dx


# softmax层
class SoftMax:
    def __init__ (self):
        self.y_hat = None
        
    def forward(self,x):
        
        self.y_hat = softmax(x)
        return self.y_hat
    
    def backward(self,labels):
        m = labels.shape[0]
        dx = self.y_hat.copy()
        dx = dx - labels
        return dx


# affine fc层
class Affine:
    def __init__(self,w,b):
        self.w = w # shape is (n_x,n_unit)
        self.b  = b  # shape is(1,n_unit)
        self.X = None
        self.origin_x_shape = None
        
        self.dw = None
        self.db = None
        
        self.out_shape =None
        
    def forward(self,x):
        self.origin_x_shape = x.shape 
        # 将四维的xreshape成为2维的，batch在第一维
        self.x = x.reshape(x.shape[0],-1)#(m,n)
        out =  np.dot(self.x, self.w)+self.b
        self.out_shape = out.shape
        return out
    
    def backward(self,dout,learning_rate = 0.01):
        m = float(self.x.shape[0])
        
        self.dw = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0,keepdims=True)
 
        dx = np.dot(dout,self.w.T)
        
        dx = dx.reshape(self.origin_x_shape) # 保持与之前的x一样的shape
        
        #更新W和b
        self.w = self.w-learning_rate*self.dw/m
        self.b = self.b - learning_rate*self.db/m
        return dx

# model
class SimpleConvNet:

    def __init__(self):
        self.X = None
        self.Y= None
        self.layers = []

    def add_conv_layer(self, core_num, cha_num, f, stride=1, pad=0):
        # 初始化W，b
        w = np.random.randn(core_num, cha_num, f, f)*0.1
        bias = np.zeros((1, core_num))
        # 卷积层
        Conv = Convolution(w, bias, stride=stride, pad=pad)
        return Conv

    def add_maxpool_layer(self, pool_shape, stride=1, pad=0):
        pool_h, pool_w = pool_shape
        pool = Pooling(pool_h, pool_w, stride=stride, pad=pad)
        
        return pool
    
    def add_affine(self, m, n_units):
        w= np.random.randn(m, n_units)*0.1
        
        b = np.zeros((1, n_units))
        
        fc_layer = Affine(w,b)
        
        return fc_layer
    
    def add_relu(self):
        relu_layer =  Relu()
        return relu_layer
    
    
    def add_softmax(self):
        softmax_layer = SoftMax()
        return softmax_layer
    
    #计算卷积或池化后的H和W
    def cacl_out_hw(self,HW,f,stride = 1,pad = 0): # 卷积核的大小
        return (HW+2*pad - f)/stride+1
    

    
    
    def init_model(self,train_X,n_classes,continue_train = False):
        save_index = []
        N,C,H,W = train_X.shape

        #卷积层 卷积核大小为5*5，步长是1，一共有6个卷积核
        n_filter = 6
        f = 5
        conv_layer = self.add_conv_layer(n_filter,C,f,stride=1)
        out_h = self.cacl_out_hw(H,f)
        out_w = self.cacl_out_hw(W,f)
        out_ch = n_filter
        self.layers.append(conv_layer) # layer记录层是什么
        save_index.append(0)

        #池化 池化核大小为2*2，步长是2，最大池化
        f = 2
        pool_layer = self.add_maxpool_layer(pool_shape=(f,f),stride=2) #aaa stride =2 原来
        out_h = self.cacl_out_hw(out_h,f,stride=2)
        out_w = self.cacl_out_hw(out_w,f,stride=2)
        self.layers.append(pool_layer)
        
        #卷积层2 卷积核大小为5*5，步长1，一共有16个卷积核
        n_filter = 16
        f = 5
        conv_layer2 = self.add_conv_layer(n_filter,out_ch,f,stride=1)
        out_h = self.cacl_out_hw(out_h,f)
        out_w = self.cacl_out_hw(out_w,f)
        out_ch = n_filter
        self.layers.append(conv_layer2) # layer记录层是什么
        save_index.append(2)
        
        # relu
        relu_layer = self.add_relu()
        self.layers.append(relu_layer) 
        

        #池化2 池化核大小为2*2，步长是2，最大池化
        f = 2
        pool_layer2 = self.add_maxpool_layer(pool_shape=(f,f),stride=2) #aaa stride =2 原来
        out_h = self.cacl_out_hw(out_h,f,stride=2)
        out_w = self.cacl_out_hw(out_w,f,stride=2)
        self.layers.append(pool_layer2)

        #FC
        n_units=int(out_h*out_w*out_ch)
        fc_layer = self.add_affine(n_units,n_classes)
        self.layers.append(fc_layer)
        save_index.append(5)

        #SoftMax
        softmax_layer = self.add_softmax()
        self.layers.append(softmax_layer)
        self.save_index = save_index
        if continue_train :
            save=np.load("D:\\torch_work\\cnn_param.npy",allow_pickle= True)
            i = 0
            for index in save_index:
                self.layers[index].w = save[i]
                i = i+1
                self.layers[index].b = save[i]
                i = i+1
        
    def forward_progation(self,train_X, print_out = False): # 优化 用一个循环就可以了 
        N,C,H,W = train_X.shape
        index = 0
        X = train_X
        for index in range(len(self.layers)):
            layer = self.layers[index]
            X = layer.forward(X)
            if print_out:
                print(str(index)+"次操作之后："+str(X.shape))
        
        return X
        
    def back_progation(self,train_y,learning_rate = 0.01):
        save =[]
        dout = train_y
        for index in range(len(self.layers)-1,-1,-1):
            layer = self.layers[index]
            dout = layer.backward(dout)
        for index in self.save_index:
            save.append(self.layers[index].w)
            save.append(self.layers[index].b)
        save = np.array(save)
        np.save("D:\\torch_work\\cnn_param",save)
        
      
    def get_minibatch(self,batch_data,minibatch_size,num):
        m_examples = batch_data.shape[0]
        minibatches = math.ceil( m_examples / minibatch_size) # “向上取整”
 
        if(num < minibatches):
            return batch_data[num*minibatch_size:(num+1)*minibatch_size]
        else:
            return batch_data[num*minibatch_size:m_examples]
    
    
    def optimize(self,train_x, train_y,test_x, test_y, minibatch_size,learning_rate=0.01,num_iters=500):

        m = train_x.shape[0]
        num_batches  = math.ceil(m / minibatch_size)

        train_losss = [] # 训练集上的loss
        test_losss  = []
        train_accs  = [] # 训练集上的acc
        test_accs   = []
        
        for iteration in range(num_iters):
            for batch_num in range(num_batches):
                minibatch_x = self.get_minibatch(train_x,minibatch_size,batch_num)
                minibatch_y = self.get_minibatch(train_y,minibatch_size,batch_num)
                # 前向传播
                A = self.forward_progation(minibatch_x)
                # 反向传播
                self.back_progation(minibatch_y,learning_rate)
                acc = (np.argmax(A,axis = 1) == np.argmax(minibatch_y, axis = 1)).sum() / minibatch_size
                loss = cross_entropy(minibatch_y, A) / minibatch_size
                print("iter : %d, batch : %d, acc %g, loss %g "%(iteration, batch_num, acc, loss))
            y_pred     = self.forward_progation(train_x)
            y_pred_t   = self.forward_progation(test_x)
            train_acc  = (np.argmax(y_pred,axis = 1) == np.argmax(train_y, axis = 1)).sum() / train_x.shape[0]
            test_acc   = (np.argmax(y_pred_t,axis = 1) == np.argmax(test_y, axis = 1)).sum() / test_x.shape[0]
            train_loss = cross_entropy(train_y, y_pred) / train_x.shape[0]
            test_loss  = cross_entropy(test_y, y_pred_t) / test_x.shape[0]
            print(f"\nTurn: {iteration} -->\n"
              f"Train Loss: {train_loss}\n"
              f"Test Loss: {test_loss}\n"
              f"Train Accuracy: {train_acc}\n"
              f"Test Accuracy: {test_acc}\n")
            train_losss.append(train_loss)
            test_accs.append(test_acc)
            train_accs.append(train_acc)
            test_losss.append(test_loss)
            
        #画出损失函数图
        plt.figure(1)
        plt.plot(train_losss,color='b')
        plt.plot(test_losss,color='r')
        plt.xlabel("iterations")
        plt.ylabel("loss")
        plt.savefig('cnn_loss_r.png')

        #acc图
        plt.figure(2)
        plt.plot(train_accs,color='b')
        plt.plot(test_accs,color='r')
        plt.xlabel("iterations")
        plt.ylabel("acc")
        plt.savefig('cnn_acc_r.png')
        
       
    def predicate(self, train_X):
        """
        预测
        """
        logits = self.forward_progation(train_X)
        one_hot = np.zeros_like(logits) # 生成logits大小的zero矩阵
        one_hot[range(train_X.shape[0]),np.argmax(logits,axis=1)] = 1
        return one_hot   

    def fit(self,train_X, train_y, test_x, test_y):
        """
        训练
        """
        self.X = train_X
        self.Y = train_y
        n_y = train_y.shape[1]
        m = train_X.shape[0]
        
        #初始化模型
        self.init_model(train_X,n_classes=n_y,continue_train = False)
        self.optimize(train_X, train_y, test_x, test_y,minibatch_size=600,learning_rate=0.01,num_iters=10)
        
        logits = self.predicate(train_X)
        
        accuracy = np.sum(np.argmax(logits,axis=1) == np.argmax(train_y,axis=1))/m
        print("训练集的准确率为：%g" %(accuracy))

In [2]:
# fit
convNet = SimpleConvNet()
convNet.fit(x_train,y_train,x_test,y_test)

iter : 0, batch : 0, acc 0.191667, loss 2.29274 
iter : 0, batch : 1, acc 0.175, loss 2.29938 
